In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=e7338a566567062f92632e87e5ff36deb2d240cc8c3147b0ca40fe7d4d119ff2
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [34]:
import pyspark
from pyspark.sql import SparkSession 
from pyspark.sql.functions import *# col,lower,regexp_replace,split
from pyspark.ml.feature import CountVectorizer, StopWordsRemover, RegexTokenizer,Tokenizer


In [4]:
sc= SparkSession.builder\
    .appName('text_analysis')\
    .getOrCreate()

In [5]:
# File location and type
file_location = "/content/sample_data/wiki_data_20thCentury.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"

df = sc.read.format(file_type)\
      .option("inferSchema", infer_schema)\
      .option("header", first_row_is_header)\
      .option("escape",'\n')\
      .option("multiLine","True")\
      .option("wholeFile","True")\
      .option("parserLib", "univocity")\
      .option("delimiter",",")\
      .load(file_location)



In [6]:
# File location and type
file_location = "/content/sample_data/us_presidents.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"

us_presidents = sc.read.format(file_type)\
      .option("inferSchema", infer_schema)\
      .option("header", first_row_is_header)\
      .option("escape",'\n')\
      .option("multiLine","True")\
      .option("wholeFile","True")\
      .option("parserLib", "univocity")\
      .option("delimiter",",")\
      .load(file_location)



In [7]:
us_presidents.show()

+---+-----+--------------+------------------+------------+--------------------+--------------------+--------------------+--------------------+
|_c0|S.No.|         start|               end|elected_year|           president|               prior|               party|                vice|
+---+-----+--------------+------------------+------------+--------------------+--------------------+--------------------+--------------------+
|  0|    1|April 30, 1789|     March 4, 1797|        null|   George Washington|Commander-in-Chie...|  Nonpartisan   [13]|          John Adams|
|  1|    2| March 4, 1797|     March 4, 1801|        null|          John Adams|1st  Vice Preside...|          Federalist|    Thomas Jefferson|
|  2|    3| March 4, 1801|     March 4, 1809|        null|    Thomas Jefferson|2nd  Vice Preside...|Democratic-  Repu...|          Aaron Burr|
|  3|    4| March 4, 1809|     March 4, 1817|        null|       James Madison|5th  United State...|Democratic-  Repu...|      George Clinton|

In [8]:
# elected_year is the primary key, foreign key and will be used to join the tables.
us_presidents=us_presidents.withColumn("elected_year",split(col('start')," ")
              .getItem(2)
              .cast('Integer')
              )
us_presidents.show()

+---+-----+--------------+------------------+------------+--------------------+--------------------+--------------------+--------------------+
|_c0|S.No.|         start|               end|elected_year|           president|               prior|               party|                vice|
+---+-----+--------------+------------------+------------+--------------------+--------------------+--------------------+--------------------+
|  0|    1|April 30, 1789|     March 4, 1797|        1789|   George Washington|Commander-in-Chie...|  Nonpartisan   [13]|          John Adams|
|  1|    2| March 4, 1797|     March 4, 1801|        1797|          John Adams|1st  Vice Preside...|          Federalist|    Thomas Jefferson|
|  2|    3| March 4, 1801|     March 4, 1809|        1801|    Thomas Jefferson|2nd  Vice Preside...|Democratic-  Repu...|          Aaron Burr|
|  3|    4| March 4, 1809|     March 4, 1817|        1809|       James Madison|5th  United State...|Democratic-  Repu...|      George Clinton|

In [9]:
# filtering the dateset, since we have information for only 19th century.
us_presidents = us_presidents.filter(us_presidents.elected_year>=1900)
us_presidents

DataFrame[_c0: int, S.No.: int, start: string, end: string, elected_year: int, president: string, prior: string, party: string, vice: string]

In [10]:
# since the elections dataset has +/-1 for elected_year.
us_presidents=us_presidents.withColumn('elected_year_1',col('elected_year')%4)
us_presidents.show()

+---+-----+------------------+-----------------+------------+--------------------+--------------------+----------+------------------+--------------+
|_c0|S.No.|             start|              end|elected_year|           president|               prior|     party|              vice|elected_year_1|
+---+-----+------------------+-----------------+------------+--------------------+--------------------+----------+------------------+--------------+
| 25|   26|September 14, 1901|    March 4, 1909|        1901|  Theodore Roosevelt|25th  Vice Presid...|Republican|     Office vacant|             1|
| 26|   27|     March 4, 1909|    March 4, 1913|        1909| William Howard Taft|42nd  United Stat...|Republican|  James S. Sherman|             1|
| 27|   28|     March 4, 1913|    March 4, 1921|        1913|      Woodrow Wilson|34th  Governor of...|Democratic|Thomas R. Marshall|             1|
| 28|   29|     March 4, 1921|   August 2, 1923|        1921|   Warren G. Harding|U.S. Senator   ( ...|Rep

In [11]:
us_presidents=us_presidents\
.withColumn('e_year',
            when(us_presidents.elected_year_1 ==1,us_presidents.elected_year-1)
            .when(us_presidents.elected_year_1==3,us_presidents.elected_year+1)
            .otherwise(us_presidents.elected_year)
            )\
.drop(us_presidents.elected_year)\
.drop(us_presidents.elected_year_1)\
.withColumnRenamed("e_year", "elected_year")


In [12]:
us_presidents.show()

+---+-----+------------------+-----------------+--------------------+--------------------+----------+------------------+------------+
|_c0|S.No.|             start|              end|           president|               prior|     party|              vice|elected_year|
+---+-----+------------------+-----------------+--------------------+--------------------+----------+------------------+------------+
| 25|   26|September 14, 1901|    March 4, 1909|  Theodore Roosevelt|25th  Vice Presid...|Republican|     Office vacant|        1900|
| 26|   27|     March 4, 1909|    March 4, 1913| William Howard Taft|42nd  United Stat...|Republican|  James S. Sherman|        1908|
| 27|   28|     March 4, 1913|    March 4, 1921|      Woodrow Wilson|34th  Governor of...|Democratic|Thomas R. Marshall|        1912|
| 28|   29|     March 4, 1921|   August 2, 1923|   Warren G. Harding|U.S. Senator   ( ...|Republican|   Calvin Coolidge|        1920|
| 29|   30|    August 2, 1923|    March 4, 1929|     Calvin Co

The elected_year column has one issue to be handled before joining with elections table
> If a president has served for more than 1 term consecutively, the corresponding rows are not created, instead a year range is given.


In [13]:
# gets the size of the date
df.count()

30

In [14]:
#Convert to lowercase

df=df.withColumn('summary',lower(col('summary')))
df.show()

+--------------------+-------+--------------------+
|               title|country|             summary|
+--------------------+-------+--------------------+
|1900 United State...|    USA|the 1900 united s...|
|1904 United State...|    USA|"the 1904 united ...|
|1908 United State...|    USA|the 1908 united s...|
|1912 United State...|    USA|"the 1912 united ...|
|1916 United State...|    USA|"the 1916 united ...|
|1920 United State...|    USA|"the 1920 united ...|
|1924 United State...|    USA|"the 1924 united ...|
|1928 United State...|    USA|the 1928 united s...|
|1932 United State...|    USA|"the 1932 united ...|
|1936 United State...|    USA|the 1936 united s...|
|1940 United State...|    USA|the 1940 united s...|
|1944 United State...|    USA|the 1944 united s...|
|1948 United State...|    USA|"the 1948 united ...|
|1952 United State...|    USA|"the 1952 united ...|
|1956 United State...|    USA|"the 1956 united ...|
|1960 United State...|    USA|the 1960 united s...|
|1964 United

Feature Engineering

In [15]:
# Creating year column
df = df.withColumn('election_year',split(df['title']," ").getItem(0))
df.show()

+--------------------+-------+--------------------+-------------+
|               title|country|             summary|election_year|
+--------------------+-------+--------------------+-------------+
|1900 United State...|    USA|the 1900 united s...|         1900|
|1904 United State...|    USA|"the 1904 united ...|         1904|
|1908 United State...|    USA|the 1908 united s...|         1908|
|1912 United State...|    USA|"the 1912 united ...|         1912|
|1916 United State...|    USA|"the 1916 united ...|         1916|
|1920 United State...|    USA|"the 1920 united ...|         1920|
|1924 United State...|    USA|"the 1924 united ...|         1924|
|1928 United State...|    USA|the 1928 united s...|         1928|
|1932 United State...|    USA|"the 1932 united ...|         1932|
|1936 United State...|    USA|the 1936 united s...|         1936|
|1940 United State...|    USA|the 1940 united s...|         1940|
|1944 United State...|    USA|the 1944 united s...|         1944|
|1948 Unit

In [16]:
# merging both tables
combined_us_elections=df.join(us_presidents,df.election_year ==  us_presidents.elected_year,"left")
combined_us_elections.show()


+--------------------+-------+--------------------+-------------+----+-----+------------------+-----------------+--------------------+--------------------+----------+------------------+------------+
|               title|country|             summary|election_year| _c0|S.No.|             start|              end|           president|               prior|     party|              vice|elected_year|
+--------------------+-------+--------------------+-------------+----+-----+------------------+-----------------+--------------------+--------------------+----------+------------------+------------+
|1900 United State...|    USA|the 1900 united s...|         1900|  25|   26|September 14, 1901|    March 4, 1909|  Theodore Roosevelt|25th  Vice Presid...|Republican|     Office vacant|        1900|
|1904 United State...|    USA|"the 1904 united ...|         1904|null| null|              null|             null|                null|                null|      null|              null|        null|
|1908

In [17]:

elections_data = combined_us_elections.select('country','election_year','president','party','summary')
elections_data.show()

+-------+-------------+--------------------+----------+--------------------+
|country|election_year|           president|     party|             summary|
+-------+-------------+--------------------+----------+--------------------+
|    USA|         1900|  Theodore Roosevelt|Republican|the 1900 united s...|
|    USA|         1904|                null|      null|"the 1904 united ...|
|    USA|         1908| William Howard Taft|Republican|the 1908 united s...|
|    USA|         1912|      Woodrow Wilson|Democratic|"the 1912 united ...|
|    USA|         1916|                null|      null|"the 1916 united ...|
|    USA|         1920|   Warren G. Harding|Republican|"the 1920 united ...|
|    USA|         1924|     Calvin Coolidge|Republican|"the 1924 united ...|
|    USA|         1928|      Herbert Hoover|Republican|the 1928 united s...|
|    USA|         1932|Franklin D. Roose...|Democratic|"the 1932 united ...|
|    USA|         1936|                null|      null|the 1936 united s...|

In [18]:
from pyspark.sql import Window
from pyspark.sql.functions import last

# Creating the window and forward filling the missing values
window = Window.partitionBy('country')\
               .orderBy('election_year')\
               .rowsBetween(-sys.maxsize, 0) # maxsize can be replace with 3 in this case, since the max consecutive nulls in the data is 3

# president column
president_na = last(elections_data['president'], ignorenulls=True).over(window)

# imputing missing values
president_na_filled = elections_data.withColumn('elected_president', president_na)

# Party column
party_na = last(president_na_filled['party'], ignorenulls=True).over(window)
elections_df = president_na_filled.withColumn('candidate_party', party_na)

elections_df = elections_df.drop('president','party')
elections_df.show()      

+-------+-------------+--------------------+--------------------+---------------+
|country|election_year|             summary|   elected_president|candidate_party|
+-------+-------------+--------------------+--------------------+---------------+
|    USA|         1900|the 1900 united s...|  Theodore Roosevelt|     Republican|
|    USA|         1904|"the 1904 united ...|  Theodore Roosevelt|     Republican|
|    USA|         1908|the 1908 united s...| William Howard Taft|     Republican|
|    USA|         1912|"the 1912 united ...|      Woodrow Wilson|     Democratic|
|    USA|         1916|"the 1916 united ...|      Woodrow Wilson|     Democratic|
|    USA|         1920|"the 1920 united ...|   Warren G. Harding|     Republican|
|    USA|         1924|"the 1924 united ...|     Calvin Coolidge|     Republican|
|    USA|         1928|the 1928 united s...|      Herbert Hoover|     Republican|
|    USA|         1932|"the 1932 united ...|Franklin D. Roose...|     Democratic|
|    USA|       

In [20]:
elections_df = elections_df\
              .withColumn('elected_president',lower(col('elected_president')))\
              .withColumn('candidate_party',lower(col('candidate_party')))
elections_df.show()

+-------+-------------+--------------------+--------------------+---------------+
|country|election_year|             summary|   elected_president|candidate_party|
+-------+-------------+--------------------+--------------------+---------------+
|    USA|         1900|the 1900 united s...|  theodore roosevelt|     republican|
|    USA|         1904|"the 1904 united ...|  theodore roosevelt|     republican|
|    USA|         1908|the 1908 united s...| william howard taft|     republican|
|    USA|         1912|"the 1912 united ...|      woodrow wilson|     democratic|
|    USA|         1916|"the 1916 united ...|      woodrow wilson|     democratic|
|    USA|         1920|"the 1920 united ...|   warren g. harding|     republican|
|    USA|         1924|"the 1924 united ...|     calvin coolidge|     republican|
|    USA|         1928|the 1928 united s...|      herbert hoover|     republican|
|    USA|         1932|"the 1932 united ...|franklin d. roose...|     democratic|
|    USA|       

In [21]:
elections_df.withColumn('summary_length',length(col('summary'))).show()

+-------+-------------+--------------------+--------------------+---------------+--------------+
|country|election_year|             summary|   elected_president|candidate_party|summary_length|
+-------+-------------+--------------------+--------------------+---------------+--------------+
|    USA|         1900|the 1900 united s...|  theodore roosevelt|     republican|          1816|
|    USA|         1904|"the 1904 united ...|  theodore roosevelt|     republican|          1499|
|    USA|         1908|the 1908 united s...| william howard taft|     republican|          1390|
|    USA|         1912|"the 1912 united ...|      woodrow wilson|     democratic|           470|
|    USA|         1916|"the 1916 united ...|      woodrow wilson|     democratic|          2116|
|    USA|         1920|"the 1920 united ...|   warren g. harding|     republican|          2771|
|    USA|         1924|"the 1924 united ...|     calvin coolidge|     republican|          1153|
|    USA|         1928|the 192

#EDA



1.   Number of presidents
2.   How many times each president is elected
3.   Number of elections won by party
4.   Presidents who have served more than 1 term



1. Number of distinct/unique values in each column

In [22]:
elections_df.agg(*(countDistinct(col(c)).alias(c) for c in elections_df.columns)).show()

+-------+-------------+-------+-----------------+---------------+
|country|election_year|summary|elected_president|candidate_party|
+-------+-------------+-------+-----------------+---------------+
|      1|           30|     30|               19|              2|
+-------+-------------+-------+-----------------+---------------+



Total number of presidents that are elected since **1900**

In [23]:
elections_df.agg(countDistinct(col("elected_president")).alias("Total_presidents_since_1900")).show()

+---------------------------+
|Total_presidents_since_1900|
+---------------------------+
|                         19|
+---------------------------+



2. Number of times each president is elected

In [24]:
pres_cnt= elections_df\
                    .groupby('elected_president')\
                    .count()
pres_cnt.show()

+--------------------+-----+
|   elected_president|count|
+--------------------+-----+
|        bill clinton|    2|
|       richard nixon|    2|
|  theodore roosevelt|    2|
| william howard taft|    1|
|dwight d. eisenhower|    2|
|        jimmy carter|    1|
|   warren g. harding|    1|
|       ronald reagan|    2|
|     harry s. truman|    2|
|     john f. kennedy|    1|
|franklin d. roose...|    3|
|   lyndon b. johnson|    1|
|      george w. bush|    2|
|        donald trump|    1|
|     calvin coolidge|    1|
|      woodrow wilson|    2|
|      herbert hoover|    1|
|   george h. w. bush|    1|
|        barack obama|    2|
+--------------------+-----+



3. Number of elections won by party

In [25]:
elections_df\
            .groupby('candidate_party')\
            .count()\
            .alias('#presidents')\
            .show()

+---------------+-----+
|candidate_party|count|
+---------------+-----+
|     republican|   16|
|     democratic|   14|
+---------------+-----+



4. Presidents who have served more than 1 term

In [26]:
pres_cnt.where(col('count')>1)\
        .sort(col('count').desc())\
        .show()

+--------------------+-----+
|   elected_president|count|
+--------------------+-----+
|franklin d. roose...|    3|
|        bill clinton|    2|
|       richard nixon|    2|
|  theodore roosevelt|    2|
|dwight d. eisenhower|    2|
|       ronald reagan|    2|
|     harry s. truman|    2|
|      george w. bush|    2|
|      woodrow wilson|    2|
|        barack obama|    2|
+--------------------+-----+



Cleaning up the summary

In [31]:
nlp_df = elections_df.withColumn('len_summary',length(col('summary')))
nlp_df.show()

+-------+-------------+--------------------+--------------------+---------------+-----------+
|country|election_year|             summary|   elected_president|candidate_party|len_summary|
+-------+-------------+--------------------+--------------------+---------------+-----------+
|    USA|         1900|the 1900 united s...|  theodore roosevelt|     republican|       1816|
|    USA|         1904|"the 1904 united ...|  theodore roosevelt|     republican|       1499|
|    USA|         1908|the 1908 united s...| william howard taft|     republican|       1390|
|    USA|         1912|"the 1912 united ...|      woodrow wilson|     democratic|        470|
|    USA|         1916|"the 1916 united ...|      woodrow wilson|     democratic|       2116|
|    USA|         1920|"the 1920 united ...|   warren g. harding|     republican|       2771|
|    USA|         1924|"the 1924 united ...|     calvin coolidge|     republican|       1153|
|    USA|         1928|the 1928 united s...|      herbert ho

In [33]:
nlp_df\
      .groupBy('candidate_party')\
      .avg('len_summary')\
      .alias('mean_len_summary')\
      .show()


+---------------+------------------+
|candidate_party|  avg(len_summary)|
+---------------+------------------+
|     republican|         1789.6875|
|     democratic|2020.7142857142858|
+---------------+------------------+



Removing punctuations, symbols from summary

In [47]:
nlp_df = nlp_df.withColumn("cleaned_summary", 
                            regexp_replace(
                                          "summary",
                                          r"""[!\"#$%&'()*+,\-.\/:;<=>?@\[\\\]^_`{|}~]"""," "
                                          )
                          )

Creating tokens for the summary

In [49]:
tokenizer = Tokenizer(inputCol='cleaned_summary',outputCol='summary_tokens')
df_tokens= tokenizer.transform(nlp_df)
df_tokens.show()

+-------+-------------+--------------------+--------------------+---------------+-----------+--------------------+--------------------+
|country|election_year|             summary|   elected_president|candidate_party|len_summary|     cleaned_summary|      summary_tokens|
+-------+-------------+--------------------+--------------------+---------------+-----------+--------------------+--------------------+
|    USA|         1900|the 1900 united s...|  theodore roosevelt|     republican|       1816|the 1900 united s...|[the, 1900, unite...|
|    USA|         1904|"the 1904 united ...|  theodore roosevelt|     republican|       1499| the 1904 united ...|[, the, 1904, uni...|
|    USA|         1908|the 1908 united s...| william howard taft|     republican|       1390|the 1908 united s...|[the, 1908, unite...|
|    USA|         1912|"the 1912 united ...|      woodrow wilson|     democratic|        470| the 1912 united ...|[, the, 1912, uni...|
|    USA|         1916|"the 1916 united ...|    

Removing stop words

In [50]:
sw_remover = StopWordsRemover(inputCol='summary_tokens',outputCol='no_stop_summary')
df_tokens = sw_remover.transform(df_tokens)
df_tokens.show()

+-------+-------------+--------------------+--------------------+---------------+-----------+--------------------+--------------------+--------------------+
|country|election_year|             summary|   elected_president|candidate_party|len_summary|     cleaned_summary|      summary_tokens|     no_stop_summary|
+-------+-------------+--------------------+--------------------+---------------+-----------+--------------------+--------------------+--------------------+
|    USA|         1900|the 1900 united s...|  theodore roosevelt|     republican|       1816|the 1900 united s...|[the, 1900, unite...|[1900, united, st...|
|    USA|         1904|"the 1904 united ...|  theodore roosevelt|     republican|       1499| the 1904 united ...|[, the, 1904, uni...|[, 1904, united, ...|
|    USA|         1908|the 1908 united s...| william howard taft|     republican|       1390|the 1908 united s...|[the, 1908, unite...|[1908, united, st...|
|    USA|         1912|"the 1912 united ...|      woodrow 

Future implementations:


1.   Can include information from other countries,starting from North American region.
2.   Extend this usecase by combining with different data; such as GDP growth, unemployment rate, crime rate etc to compare them with various presidential terms.
3.   Implement NLP algorithms on the summary like, Tf-Idf, phrase extraction.

